In [1]:
import os
import warnings
import pandas as pd
import csv
from census import Census
from us import states
from apikey import apikey
import requests

#ignore future warnings error
warnings.simplefilter(action='ignore', category=FutureWarning)

c = Census(apikey, year=2017)

In [2]:
# Open and read Texas Graduation Rate data
file = "C:/Users/May 2018/Desktop/SMU Bootcamp Doc/GroupProject/Team6/County_Grad_Data_2017.csv"
HighSchool2017_data = pd.read_csv(file)
HighSchool2017_data.head()

FileNotFoundError: File b'C:/Users/May 2018/Desktop/SMU Bootcamp Doc/GroupProject/Team6/County_Grad_Data_2017.csv' does not exist

In [4]:
# Create TX graduation data frame
HighSchool2017_dfw = HighSchool2017_data.loc[HighSchool2017_data['CNTYNAME'].isin(['DENTON COUNTY','COLLIN COUNTY','HUNT COUNTY','ROCKWALL COUNTY','KAUFMAN COUNTY','ELLIS COUNTY','JOHNSON COUNTY','DALLAS COUNTY','SOMMERVILLE COUNTY','HOOD','PARKER COUNTY','WISE COUNTY','TARRANT COUNTY'])]
HighSchool2017_dfw = pd.DataFrame(HighSchool2017_dfw)
HighSchool2017_dfw_df = HighSchool2017_dfw.filter(items=['CNTYNAME','CNTY_ALLR_GRAD','CNTY_AAR_GRAD','CNTY_ASR_GRAD',
                                                         'CNTY_HSR_GRAD','CNTY_MUR_GRAD','CNTY_NAR_GRAD','CNTY_PIR_GRAD',
                                                         'CNTY_WHR_GRAD'])
HighSchool2017_dfw_df.rename(columns={'CNTYNAME':'County Name','CNTY_ALLR_GRAD':'All Students','CNTY_AAR_GRAD':'African American',
                             'CNTY_ASR_GRAD':'Asian','CNTY_HSR_GRAD':'Hispanic','CNTY_MUR_GRAD':'Multiracial','CNTY_NAR_GRAD':'American Indian',
                                      'CNTY_PIR_GRAD':'Pacific Islander','CNTY_WHR_GRAD':'White'},inplace=True)
HighSchool2017_dfw_df

NameError: name 'HighSchool2017_data' is not defined

In [5]:
# Open and read TX crime data 
TexasCrime2017_data = pd.read_csv("C:/Users/May 2018/Desktop/SMU Bootcamp Doc/GroupProject/Team6/County_crime_data_2017.csv")
TexasCrime2017_data.head()

FileNotFoundError: File b'C:/Users/May 2018/Desktop/SMU Bootcamp Doc/GroupProject/Team6/County_crime_data_2017.csv' does not exist

In [6]:
# Create TX crime data frame
DFWCrime2017 = TexasCrime2017_data.loc[TexasCrime2017_data['Agency Name'].isin(['DENTON COUNTY','COLLIN COUNTY','HUNT COUNTY','ROCKWALL COUNTY','KAUFMAN COUNTY','ELLIS COUNTY','JOHNSON COUNTY','DALLAS COUNTY','SOMMERVILLE COUNTY','HOOD','PARKER COUNTY','WISE COUNTY','TARRANT COUNTY'])]
DFWCrime2017_df = pd.DataFrame(DFWCrime2017)
DFWCrime2017_df.columns = ["County Name","Family Violence Incidents"]
DFWCrime2017_df


NameError: name 'TexasCrime2017_data' is not defined

In [7]:
# Merge TX graduation and crime data
HSGradCrime_2017_df = pd.merge(HighSchool2017_dfw_df,DFWCrime2017_df,on='County Name')
HSGradCrime_2017_df

NameError: name 'HighSchool2017_dfw_df' is not defined

In [8]:
# Clean county names to match name style of census
County_Names = ['Collin County, Texas','Dallas County, Texas','Denton County, Texas','Ellis County, Texas','Hunt County, Texas','Johnson County, Texas','Kaufman County, Texas','Parker County, Texas','Rockwall County, Texas','Tarrant County, Texas','Wise County, Texas']
HSGradCrime_2017_df['County'] = County_Names
New_HSGradCrime_2017_df = HSGradCrime_2017_df[['County','All Students','Family Violence Incidents']] 
New_HSGradCrime_2017_df

NameError: name 'HSGradCrime_2017_df' is not defined

In [13]:
# Get DFW counties from API
census_data = c.acs5.get('NAME', geo={'for': 'county:121,085,231,397,257,139,251,113,221,367,497,439', 
                                      'in': 'state:%s' % states.TX.fips})

county_pd = pd.DataFrame(census_data)
county_pd

,NAME,county,state
0,"Hunt County, Texas",231,48
1,"Parker County, Texas",367,48
2,"Collin County, Texas",085,48
3,"Rockwall County, Texas",397,48
4,"Tarrant County, Texas",439,48
5,"Ellis County, Texas",139,48
6,"Wise County, Texas",497,48
7,"Kaufman County, Texas",257,48
8,"Denton County, Texas",121,48
9,"Dallas County, Texas",113,48


In [14]:
# Build census data frame
baseUrl = "https://api.census.gov/data/2017/acs/acs5/subject?get="
baseUrl2 = "https://api.census.gov/data/2017/acs/acsse?get="

#census ids
ids = ["K202002_001E", "K202503_002E", 
       "K202503_003E", "K201001_001E",
       "K201501_001E",  "K200101_003E",
       "K200101_002E"]

columns = ['County', 'Median Earnings', '# of Home Owners',
          '# of Renters', '# Married', "Education", 
           "Female Population", "Male Population" ]

censusDf = pd.DataFrame(columns=columns)


In [15]:
# get data for each id by county and populate dataframe
count = 0

for countyid in county_pd['county']: 
    
    for item in ids:
        targetUrl = (baseUrl2 + item + ",NAME" + "&for=county:" +
                           countyid + "&in=state:48" + "&key=" + apikey)
        resp = requests.get(targetUrl).json()
        
        if resp[0][0] == 'K202002_001E':
            censusDf.set_value(count, "Median Earnings", resp[1][0])
        if  resp[0][0] == 'K202503_002E':   
            censusDf.set_value(count, "# of Home Owners", resp[1][0])
        if  resp[0][0] == 'K202503_003E':     
            censusDf.set_value(count, "# of Renters", resp[1][0])
        if  resp[0][0] == 'K201001_001E':      
            censusDf.set_value(count, "# Married", resp[1][0])
        if  resp[0][0] == 'K201501_001E':       
            censusDf.set_value(count, "Education", resp[1][0])
        if resp[0][0] == 'K200101_003E':
            censusDf.set_value(count, "Female Population", resp[1][0])
        if resp[0][0] == 'K200101_002E':
            censusDf.set_value(count, "Male Population", resp[1][0])    
    censusDf.set_value(count, "County", resp[1][1])   
    count += 1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
censusDf.head(12)

,County,Median Earnings,# of Home Owners,# of Renters,# Married,Education,Female Population,Male Population
0,"Hunt County, Texas",33664,64565,26438,75087,61557,48201,45671
1,"Parker County, Texas",35129,106949,24700,106767,90367,66140,67323
2,"Collin County, Texas",50212,665040,301277,760920,635534,492767,476836
3,"Rockwall County, Texas",50215,81035,15124,75858,63099,49144,47644
4,"Tarrant County, Texas",36216,1300652,728517,1600866,1317559,1047730,1006745
5,"Ellis County, Texas",37201,130247,41512,135260,111660,87970,85650
6,"Wise County, Texas",30706,50042,15105,52852,44568,33054,33127
7,"Kaufman County, Texas",36480,97537,23768,94327,78533,63092,59791
8,"Denton County, Texas",43250,579415,244764,662965,547898,424882,411328
9,"Dallas County, Texas",32275,1392954,1194085,2037627,1676011,1326753,1291395


In [ ]:
# Merge all three data sets by county name
Merged_3_df = pd.merge(New_HSGradCrime_2017_df,censusDf,on='County')
Merged_3_df

In [ ]:
# Caluate new metrics ... just an example - need more
# Home Ownership Ratio (# of Home Owners / # of Renters)
Merged_3_df["Home Ownership Ratio"] = Merged_3_df["# of Home Owners"].astype(int) / Merged_3_df["# of Renters"].astype(int)
Merged_3_df


In [ ]:
# Final data frame for analysis and graphs
Final_df = Merged_3_df[['County','All Students','Family Violence Incidents','Median Earnings','Home Ownership Ratio']]

# Sets index to counties
Final_df = Final_df.set_index('County')
Final_df

In [ ]:
# Build a scatter plot: Median Earnings
plt.scatter(Final_df["Median Earnings"], Final_df["All Students"], marker="o", color="steelblue", edgecolors="black")

# Scatter plot no. 1 - graph properties
plt.title("Graduation Rate vs. Median Earnings")
plt.ylabel("Graduation Rate (County)")
plt.xlabel("Median Earnings (County)")
plt.grid(True)

# Save the figures
#plt.savefig("...GradRateVsMedIncome.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot: Median Earnings
plt.scatter(Final_df["Home Ownership Ratio"], Final_df["All Students"], marker="o", color="yellow", edgecolors="black")

# Scatter plot no. 1 - graph properties
plt.title("Graduation Rate vs. Home Ownership Ratio")
plt.ylabel("Graduation Rate (County)")
plt.xlabel("Home Ownership Ratio (County)")
plt.grid(True)

# Save the figures
#plt.savefig("...HORVsMedIncome.png")

# Show plot
plt.show()